# Configuración de motores

🟢 Motores Virtuales Conectados\
🟢 Motores Físicos Conectados\
🟢 f: Reset to zero\
🟢 f: Timed back and forth\
🟢 f: Keyboard activated Joystick\
🟢 f: Signal activated back and forth\
🔴 Stepper Rotor\
🔴 Parametrización\
🔴 In: Generador de Pulsos\
🔴 Sincronización

## Librerías

### Descarga

In [2]:
%pip install libximc
%pip install matplotlib
%pip install keyboard

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importar librerías

In [1]:
import pathlib, os, time
import matplotlib.pyplot as plt
import libximc.highlevel as ximc
import keyboard, threading

## Funciones

### Respuesta de teclado

In [2]:
def on_key_press(event):
    pass


def keyboard_listener():
    hook = keyboard.on_press(on_key_press)
    keyboard.wait('esc')
    keyboard.unhook(hook)

### Parametrización de motores

In [3]:
# A posición cero
def home(axis):
    axis.command_move(0, 0)
    axis.command_wait_for_stop(100)


def movimiento_axial(axis, pasos, sleep, direccion):
    axis.command_movr(pasos*direccion, 0)
    axis.command_wait_for_stop(sleep)


def movimiento_angular(motor, grados, velocidad, f_l_posicion):
    motor.set_speed(velocidad)
    motor.move_to(grados)
    motor.wait_for_stop()


def parametrizacion(l_axis_1, l_paso, l_sleep, l_axis_2, init, direccion, a_axis=None, a_paso=None, a_sleep=None):
    if any(param is not None for param in [a_axis, a_paso, a_sleep]) and not all(param is not None for param in [a_axis, a_paso, a_sleep]):
        raise ValueError("Si se proporcionan parámetros de movimiento angular, deben ingresarse todos: 'a_axis', 'a_paso' y 'a_sleep'.")
    if not init:
        home(l_axis_1)
        print("Axis_1 in home")
        home(l_axis_2)
        print("Axis_2 in home")
        if a_axis is not None:
            home(a_axis)
        init = True
    
    l_posicion_1 = l_axis_1.get_position().Position
    if l_posicion_1 >= 1000-l_paso and direccion == 1:
        direccion = -1
    elif l_posicion_1 <= 0+l_paso and direccion == -1:
        direccion = 1
    movimiento_axial(l_axis_1, l_paso, l_sleep, direccion)
    print(l_posicion_1)
    l_posicion_2 = l_axis_1.get_position().Position

    if a_axis is not None:
        a_posicion = a_axis.get_position().Position
        f_l_posicion = lambda x: x*l_axis_1.get_position().Position
        movimiento_angular(a_axis, a_paso, a_sleep, f_l_posicion)
    
    if l_posicion_2-l_posicion_1 > 0:
        direccion = 1
    elif l_posicion_2-l_posicion_1 < 0:
        direccion = -1
    return direccion

## A. Controlador Virtual

In [4]:
virtual_device_filename_1 = "virtual_motor_controller_1.bin"
virtual_device_file_path_1 = os.path.join(
    pathlib.Path().cwd(),
    virtual_device_filename_1
)

virtual_device_filename_2 = "virtual_motor_controller_2.bin"
virtual_device_file_path_2 = os.path.join(
    pathlib.Path().cwd(),
    virtual_device_filename_2
)


device_uri_1 = "xi-emu:///{}".format(virtual_device_file_path_1)
device_uri_2 = "xi-emu:///{}".format(virtual_device_file_path_2)

axis_1 = ximc.Axis(device_uri_1)
axis_2 = ximc.Axis(device_uri_2)

## B. Controlador físico

### Búsqueda

In [3]:
devices = ximc.enumerate_devices(
    ximc.EnumerateFlags.ENUMERATE_NETWORK |
    ximc.EnumerateFlags.ENUMERATE_PROBE
)

if len(devices) == 0:
    print("No encontrado")
else:
    # Print real devices list
    print("Found {} real device(s):".format(len(devices)))
    for device in devices:
        print("  {}".format(device))

No encontrado


### Instancia

In [4]:
# WINDOWS
device_uri_1 = r"xi-com:\\.\COM3" #URI aparece al abrir XILabs
device_uri_2 = r"xi-com:\\.\COM4" #URI aparece al abrir XILabs

# LINUX/MAC
#device_uri_1 = r"xi-com:///dev/ttyACM29"
#device_uri_2 = r"xi-com:///dev/ttyACM30"

axis_1 = ximc.Axis(device_uri_1)
axis_2 = ximc.Axis(device_uri_2)

## Demos
- A posiciones específicas
- Se va en vola

In [22]:
axis_1.open_device()

# ==== Set current position as zero ====
axis_1.command_zero()

# Object instances should be passed by reference
position = axis_1.get_position()
print("Initial position:", position.Position)


# ==== Move to the first absolute position (X = 100) ====
next_position = 100
print("Move to position:", next_position)
axis_1.command_move(next_position, 0)

print("Moving...")
axis_1.command_wait_for_stop(100)

position = axis_1.get_position()
print("Current position:", position.Position)


# ==== Move to the second absolute position (X = 50) ====
next_position = 50
print("Move to position:", next_position)
axis_1.command_move(next_position, 0)

print("Moving...")
axis_1.command_wait_for_stop(100)

position = axis_1.get_position()
print("Current position:", position.Position)


# ==== Perform a relative shift by 100 ====
relative_shift = 100
print("Perform a relative shift by", relative_shift)
print("So we are going to", position.Position, "+", relative_shift,
      " =", position.Position + relative_shift)
axis_1.command_movr(relative_shift, 0)

print("Moving...")
axis_1.command_wait_for_stop(100)

position = axis_1.get_position()
print("Current position:", position.Position)


# ==== Perform a relative shift by -150 ====
relative_shift = -150
print("Perform a relative shift by", relative_shift)
print("So we are going to", position.Position, "+ (", relative_shift,
      ") =", position.Position + relative_shift)
axis_1.command_movr(relative_shift, 0)

print("Moving...")
axis_1.command_wait_for_stop(100)

position = axis_1.get_position()
print("Current position:", position.Position)

axis_1.close_device()
print("Done")

Initial position: 0
Move to position: 100
Moving...
Current position: 100
Move to position: 50
Moving...
Current position: 50
Perform a relative shift by 100
So we are going to 50 + 100  = 150
Moving...
Current position: 141
Perform a relative shift by -150
So we are going to 141 + ( -150 ) = -9
Moving...
Current position: -8
Done


In [23]:
axis_2.open_device()

positions = []
timestamps_s = []

# ==== Shift the stage ====
next_position = 4000
print("Move to position:", next_position)
axis_2.command_move(next_position, 0)

print("Moving...")

t_start_s = time.time()

# We will move for 2 s with current speed. Then we will increase the speed.
while (time.time() < t_start_s + 2):
    position = axis_2.get_position()

    # Do some job during the motion
    positions.append(position.Position)
    timestamps_s.append(time.time() - t_start_s)

    time.sleep(0.1)

    # Update status for the next check of the loop condition
    statux = axis_2.get_status()


# Increase the speed on the fly
move_settings = axis_2.get_move_settings()
move_settings.Speed *= 4
move_settings.Accel *= 4
move_settings.Decel *= 4
axis_2.set_move_settings(move_settings)
print("Speed increased 4 times")

# Wait until the end of the motion
status = axis_2.get_status()
while status.MvCmdSts & ximc.MvcmdStatus.MVCMD_RUNNING:
    position = axis_2.get_position()

    # Do some job during the motion
    positions.append(position.Position)
    timestamps_s.append(time.time() - t_start_s)

    time.sleep(0.1)

    # Update status for the next check of the loop condition
    status = axis_2.get_status()


# ==== Shift the stage back ====
# Decrease the speed
move_settings = axis_2.get_move_settings()
move_settings.Speed //= 8
axis_2.set_move_settings(move_settings)
print("Speed decreased 8 times")

next_position = 0
print("Move to position:", next_position)
axis_2.command_move(next_position, 0)

print("Moving...")

# Check axis_1 motion status in a loop
status = axis_2.get_status()
while status.MvCmdSts & ximc.MvcmdStatus.MVCMD_RUNNING:
    position = axis_1.get_position()

    # Do some job during the motion
    positions.append(position.Position)
    timestamps_s.append(time.time() - t_start_s)

    time.sleep(0.1)

    # Update status for the next check of the loop condition
    status = axis_1.get_status()


print("Movement finished")


print("Set initial speed")
move_settings = axis_1.get_move_settings()
move_settings.Speed *= 2
move_settings.Accel //= 4
move_settings.Decel //= 4
axis_2.set_move_settings(move_settings)

axis_2.close_device()

# ==== Plot motion profile ====
plt.plot(timestamps_s, positions)
plt.axvline(2, linestyle="dotted")
plt.xlabel("Time, s")
plt.ylabel("Position, steps")
plt.show()

ConnectionError: Cannot connect to device via URI='xi-com:\\.\COM4'
	* check URI. For URI format see documentation for open_device() on https://libximc.xisupport.com/doc-en/
	* check whether it's connected to computer physically, powered and not occupied by another app


## A lo que nos compete

### Timed Back and Forth

In [6]:
# Move axis_1 to position 0
axis_1.open_device()
axis_1.command_move(0, 0)
axis_1.command_wait_for_stop(100)
init_position = axis_1.get_position().Position
print("Initial position:", init_position)
position = 200
# Move axis_1 to position 5000 with step 300
while position <= 5000:
    axis_1.command_move(position, 0)
    axis_1.command_wait_for_stop(100)
    position += 300
    print("Current position:", axis_1.get_position().Position)

# Move axis_1 to position 0 with step 300
while position >= 300:
    axis_1.command_move(position, 0)
    axis_1.command_wait_for_stop(100)
    position -= 300
    print("Current position:", axis_1.get_position().Position)
axis_1.command_move(0, 0)
axis_1.close_device()

Initial position: 0
Current position: 200
Current position: 500
Current position: 800
Current position: 1100
Current position: 1400
Current position: 1700
Current position: 2000
Current position: 2300
Current position: 2600
Current position: 2900
Current position: 3200
Current position: 3500
Current position: 3800
Current position: 4100
Current position: 4400
Current position: 4700
Current position: 5000
Current position: 5300
Current position: 5000
Current position: 4700
Current position: 4400
Current position: 4100
Current position: 3800
Current position: 3500
Current position: 3200
Current position: 2900
Current position: 2600
Current position: 2300
Current position: 2000
Current position: 1700
Current position: 1400
Current position: 1100
Current position: 800
Current position: 500


### Joystick

In [5]:
listener = threading.Thread(target=keyboard_listener)
listener.start()

axis_1.open_device()
axis_2.open_device()
while listener.is_alive():
    if keyboard.is_pressed('up'):
        axis_1.command_movr(100, 0)
        axis_1.command_wait_for_stop(100)
        print("up")
    if keyboard.is_pressed('down'):
        axis_1.command_movr(-100, 0)
        axis_1.command_wait_for_stop(100)
        print("down")
    if keyboard.is_pressed('right'):
        axis_2.command_movr(100, 0)
        axis_2.command_wait_for_stop(100)
        print("right")
    if keyboard.is_pressed('left'):
        axis_2.command_movr(-100, 0)
        axis_2.command_wait_for_stop(100)
        print("left")
axis_1.close_device()
axis_2.close_device()

left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
left
left
left
left
left
left
left
left
left
left
left
left
left
down
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up


### Signal based Back and Forth

In [8]:
listener = threading.Thread(target=keyboard_listener)
listener.start()
a_axis = 1
axis_1.open_device()
axis_2.open_device()
init = False
while listener.is_alive():
    if keyboard.is_pressed('f'):
        if not init:
            direccion_anterior = parametrizacion(l_axis_1=axis_1, l_paso=100, l_sleep=100, l_axis_2=axis_2, init=init, direccion=1)
            init = True
        else:
            direccion_nueva = parametrizacion(l_axis_1=axis_1, l_paso=100, l_sleep=100, l_axis_2=axis_2, init=init, direccion = direccion_anterior)
            direccion_anterior = direccion_nueva
axis_1.close_device()
axis_2.close_device()

Axis_1 in home
Axis_2 in home
0
100
200
300
400
500
600
700
800
900
800
700
600
500
400
300
200
100
200
300
400
500
600
700
800
900
800
700
600
500
400
300
200
100
200
300
400
500
600
700
800
900
800
700
600
500
400
300
200
100
200
300
400
500
600
700
800
900
